# Fetching the Data

In [4]:
import csv
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan, ScanError

## Setup

In [16]:
# These parameters are only placeholders
url = 'http://localhost:9200'
scheme = 'https'
port = 443
pid = '123abc'
index = "elastic_index"

sessions_file = "sessions_raw.csv"
events_file = "events_raw.csv"

es = Elasticsearch(
            [url],
            scheme=scheme,
            port=port)

### ElasticSearch queries

In [10]:
session_query = {
    "query": {
        "bool": {
            "filter": [
                {
                    "term": {
                        "pid": pid
                    }
                },
                {
                    "term": {
                        "type": "session"
                    }
                },
                {
                    "range": {
                        "timeStart": {
                            "gte": "2022-04-01T00:00:00.000Z",
                            "lte": "2022-04-02T00:00:00.000Z"
                        }
                    }
                }
            ]
        }
    }
}

event_query = {
    "query": {
        "bool": {
            "filter": [
                {
                    "term": {
                        "pid": pid
                    }
                },
                {
                    "term": {
                        "type": "event"
                    }
                },
                {
                    "has_parent": {
                        "parent_type": "record",
                        "query": {
                            "bool": {
                                "filter": [
                                    {
                                        "term": {
                                            "pid": pid
                                        }
                                    },
                                    {
                                        "has_parent": {
                                            "parent_type": "session",
                                            "query": {
                                                "bool": {
                                                    "filter": [
                                                        {
                                                            "term": {
                                                                "pid": pid
                                                            }
                                                        },
                                                        {
                                                            "range": {
                                                                "timeStart": {
                                                                    "gte": "2022-04-01T00:00:00.000Z",
                                                                    "lte": "2022-04-02T00:00:00.000Z"
                                                                }
                                                            }
                                                        }
                                                    ]
                                                }
                                            }
                                        }
                                    }
                                ]
                            }
                        }
                    }
                }
            ]
        }
    }
}

### Data fields to select

In [11]:
session_fields = [
    "id",
    "uid",
    "timeStart",
    "timeClose",
    "duration",
    "pageReferer",
    "browserCode",
    "platformCode",
    "countryCode",
    "countryCity",
    "deviceType",
    "deviceOrientation",
    "deviceWidth",
    "deviceHeight",
    "pageFirst",
    "pageLast",
    "pageUrl",
]

event_fields = [
    "sid",
    "eventType",
    "time",
    "pageUrl",
]

### Helper formatting function

In [12]:
def format_row(hit, fields):
    row = []
    source = hit["_source"]
    for field in fields:
        if field == "id":
            if "_id" not in hit:
                return None
            row.append(hit["_id"])
        elif field in source:
            row.append(source[field])
        else:
            row.append(None)

    return row

## Fetch and Store

### Sessions

In [14]:
try:
    sessions = scan(es, query=session_query, doc_type="_doc", index=index, scroll="20m", request_timeout=60)
except ScanError:
    print('Scan Error')
except SystemExit:
    print('Run has been terminated')
    raise SystemExit()

counter = 0
with open(sessions_file, "a") as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=";")
    csvwriter.writerow(session_fields)

    for session in sessions:
        if session is None:
            continue
        csvwriter.writerow(format_row(session, session_fields))
        counter += 1

print("Saved {} elements into '{}' file".format(counter, sessions_file))

Saved 6246 elements into 'sessions_raw.csv' file


### Events

In [15]:
try:
    events = scan(es, query=event_query, doc_type="_doc", index=index, scroll="20m", request_timeout=60)
except ScanError:
    print('Scan Error')
except SystemExit:
    print('Run has been terminated')
    raise SystemExit()

counter = 0
with open(events_file, "a") as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=";")
    csvwriter.writerow(event_fields)

    for event in events:
        if event is None:
            continue
        csvwriter.writerow(format_row(event, event_fields))
        counter += 1

print("Saved {} elements into '{}' file".format(counter, events_file))

Saved 244889 elements into 'events_raw.csv' file
